<a href="https://colab.research.google.com/github/Madhavan12103/-Chess-Game-Result-Prediction-Using-Machine-Learning/blob/main/chess_model_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install python-chess scikit-learn pandas joblib
from google.colab import files
uploaded = files.upload()  # Select all your .pgn files when prompted
import chess.pgn
import pandas as pd
import io

def extract_features(pgn_text):
    games = []
    pgn_io = io.StringIO(pgn_text)

    while True:
        game = chess.pgn.read_game(pgn_io)
        if game is None:
            break

        try:
            result = game.headers["Result"]
            white_elo = int(game.headers.get("WhiteElo", 1500))
            black_elo = int(game.headers.get("BlackElo", 1500))
            num_moves = len(list(game.mainline_moves()))

            if result == "1-0":
                label = "White"
            elif result == "0-1":
                label = "Black"
            elif result == "1/2-1/2":
                label = "Draw"
            else:
                continue

            games.append({
                "white_elo": white_elo,
                "black_elo": black_elo,
                "num_moves": num_moves,
                "result": label
            })
        except:
            continue

    return pd.DataFrame(games)

# 📚 Loop through uploaded files
all_games = pd.DataFrame()
for filename in uploaded.keys():
    with open(filename, 'r', encoding='utf-8') as f:
        pgn_text = f.read()
        df = extract_features(pgn_text)
        all_games = pd.concat([all_games, df], ignore_index=True)

print(f"✅ Total games loaded: {len(all_games)}")
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

# Clean and prepare
all_games.dropna(inplace=True)
X = all_games[["white_elo", "black_elo", "num_moves"]]
y = all_games["result"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Save model
joblib.dump(model, "chess_master_model.pkl")
print("✅ Model saved as 'chess_master_model.pkl'")
files.download("chess_master_model.pkl")


Saving Abdusattorov.pgn to Abdusattorov.pgn
Saving Anand.PGN to Anand.PGN
Saving Carlsen.pgn to Carlsen.pgn
Saving Caruana.pgn to Caruana.pgn
Saving Erigaisi.pgn to Erigaisi.pgn
Saving Gukesh.pgn to Gukesh.pgn
Saving Nakamura.pgn to Nakamura.pgn
Saving Praggnanandhaa.pgn to Praggnanandhaa.pgn
✅ Total games loaded: 35452
              precision    recall  f1-score   support

       Black       0.60      0.57      0.58      2123
        Draw       0.53      0.50      0.51      2238
       White       0.64      0.69      0.66      2730

    accuracy                           0.59      7091
   macro avg       0.59      0.59      0.59      7091
weighted avg       0.59      0.59      0.59      7091

✅ Model saved as 'chess_master_model.pkl'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>